In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch

In [17]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import pandas as pd

# Load your data
df1=pd.read_excel('Winter23_logs.xlsx').drop(0)

df1['Outcome']=df1['Outcome'].str.lower()
df1.head()
df=pd.DataFrame({'Outcome': df1['Outcome']})
df=df.dropna()



In [18]:
df.head()

,Outcome
10,yes
11,re-enable
14,whoever pressed the button hightailed it out o...
15,it's a class - yes
20,co-pilot lost track of time doing install work


In [24]:
# Create a language modeling dataset
text_data = list(df['Outcome'])
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenized_data = tokenizer(text_data, return_tensors="pt", padding=True, truncation=True)

# Fine-tune the language model (GPT-2)
config = GPT2Config.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2", config=config)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Extract input_ids
input_ids = tokenized_data['input_ids']

# Assuming you have a train-validation split
input_ids_train, input_ids_val = train_test_split(
    input_ids, test_size=0.1, random_state=42
)

# Create DataLoader
train_data = torch.utils.data.TensorDataset(input_ids_train)
val_data = torch.utils.data.TensorDataset(input_ids_val)

train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
val_loader = DataLoader(val_data, batch_size=4, shuffle=False)


# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(3):  # You may need more epochs based on your data size and complexity
    model.train()
    for batch in tqdm(train_loader, desc="Training"):
        optimizer.zero_grad()
        inputs = {"input_ids": batch[0].to(device)}
       # inputs = {key: value.to(device) for key, value in batch.items()}
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Save the fine-tuned language model
model.save_pretrained("fine_tuned_language_model")

# Load the fine-tuned language model for classification
model = GPT2LMHeadModel.from_pretrained("fine_tuned_language_model")

Training:   0%|                                                                                | 0/767 [00:08<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 804.60 MiB already allocated; 0 bytes free; 868.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF